In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [6]:
import src.config as config
from src.inference import (
    get_feature_store
)

import pytz
from datetime import datetime, timezone, timedelta

def get_current_est_hour_ceiled():
    """Get current EST time ceiled to the next hour"""
    current = datetime.now(pytz.timezone("US/Eastern"))
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_est_hour_ceiled()
print(f"Current EST hour (ceiled): {current_hour}")

Current EST hour (ceiled): 2025-03-02 21:00:00-05:00


In [12]:
def fetch_predictions():
    current_hour = pd.Timestamp.now(tz='US/Eastern').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [13]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-02 21:00:06,338 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 21:00:06,338 INFO: Initializing external client
2025-03-02 21:00:06,347 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 21:00:07,215 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635


In [14]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.42s) 


In [15]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,190,0.0,2025-03-03 02:00:00+00:00
1,18,0.0,2025-03-03 02:00:00+00:00
2,150,0.0,2025-03-03 02:00:00+00:00
3,113,1.0,2025-03-03 02:00:00+00:00
4,106,0.0,2025-03-03 02:00:00+00:00
...,...,...,...
251,15,0.0,2025-03-03 02:00:00+00:00
252,227,0.0,2025-03-03 02:00:00+00:00
253,64,0.0,2025-03-03 02:00:00+00:00
254,115,0.0,2025-03-03 02:00:00+00:00


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  256 non-null    int32                  
 1   predicted_demand    256 non-null    float64                
 2   pickup_hour         256 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 5.1 KB


In [16]:
df["pickup_hour"] = df["pickup_hour"].dt.tz_convert("US/Eastern")
df

,pickup_location_id,predicted_demand,pickup_hour
0,190,0.0,2025-03-02 21:00:00-05:00
1,18,0.0,2025-03-02 21:00:00-05:00
2,150,0.0,2025-03-02 21:00:00-05:00
3,113,1.0,2025-03-02 21:00:00-05:00
4,106,0.0,2025-03-02 21:00:00-05:00
...,...,...,...
251,15,0.0,2025-03-02 21:00:00-05:00
252,227,0.0,2025-03-02 21:00:00-05:00
253,64,0.0,2025-03-02 21:00:00-05:00
254,115,0.0,2025-03-02 21:00:00-05:00


In [34]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current EST time and round up to next hour
    now = datetime.now(pytz.timezone("US/Eastern"))
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]
    df_hour["pickup_hour"] = df["pickup_hour"].dt.tz_convert("US/Eastern")

    print(f"Current EST time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-02 21:08:42,067 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 21:08:42,075 INFO: Initializing external client
2025-03-02 21:08:42,077 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 21:08:42,927 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Current EST time: 2025-03-02 21:08:42.067513-05:00
Next hour: 2025-03-02 22:00:00-05:00
Querying for date range: 2025-03-02 to 2025-03-03
Filtering for hour: 2025-03-02 22:00
Found 256 records


In [24]:
now = datetime.now(pytz.timezone("US/Eastern"))

In [33]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,190,0.0,2025-03-02 21:00:00-05:00
1,18,0.0,2025-03-02 21:00:00-05:00
2,150,0.0,2025-03-02 21:00:00-05:00
3,113,1.0,2025-03-02 21:00:00-05:00
4,106,0.0,2025-03-02 21:00:00-05:00
...,...,...,...
251,15,0.0,2025-03-02 21:00:00-05:00
252,227,0.0,2025-03-02 21:00:00-05:00
253,64,0.0,2025-03-02 21:00:00-05:00
254,115,0.0,2025-03-02 21:00:00-05:00
